# Representations, LDA and topics in CORD-19

In this notebook, representations of the papers will be calculated based on their text content.
Then, from these representations, a modeling of topics will be carried out using the LDA method.

Finally, the most relevant papers for each topic will be determined using the PageRank scores of each paper.

In [ ]:
!pip install -q -r requirements.txt

In [ ]:
from risotto.references import load_papers_from_metadata_file, build_papers_reference_graph, paper_as_markdown
from fastprogress.fastprogress import progress_bar

from pathlib import Path

import scispacy
import en_core_sci_sm
import networkx as nx
from collections import defaultdict
import numpy as np

Loading paper dataset and re-generating the graph of papers and the corresponding PageRank.

In [ ]:
cord19_dataset_folder = "./datasets/CORD-19-research-challenge"

In [ ]:
papers, _ = load_papers_from_metadata_file(cord19_dataset_folder)

In [ ]:
G = build_papers_reference_graph(papers)

In [ ]:
pageranks = nx.pagerank(G)

## Paper representations

In order to build a representation for each paper, the following libraries will be used:

- spaCy: https://spacy.io/
- scispaCy: https://allenai.github.io/scispacy/

The language model named`en_core_sci_sm` will be used, which has been trained with a corpus of biomedical text with a vocabulary of more than 100.000 words.
In case of needing a model with a larger vocabulary, there are some others available.

In [ ]:
# Load the biomedical language pipeline
nlp = en_core_sci_sm.load()

In [ ]:
# Select a paper to showcase spacy's features
sample_paper = list(pageranks.keys())[0]
sample_text = "\n".join([ paragraph["text"] for paragraph in sample_paper._file_contents["body_text"]])
sample_text

doc = nlp(sample_text, disable=["tagger", "parser", "ner"])
doc[17].lemma_

'be'

The document tokenized by the `spacy` pipeline is displayed.
An interesting thing about using `spacy` with the pretrained language model is that it automatically computes document and token representations vectors.
It's a pending task to find out which language model architecture it's used to compute those vectors.

A relevant aspect that influences downstream tasks is the number of out--of--vocabulary (OOV) tokens.
The following cell makes a quick inspection over a sample paper counting the number of OOV tokens.
A continuación, se realizará una iteración sobre los tokens para detectarlos.

In [ ]:
num_oov = 0
for token in doc:
    if token.is_oov and token.string != "\n":
        if token.string.endswith("virus"):
            print(token, "not found")
        num_oov += 1
    else:
        if token.string.endswith("virus"):
            print(token, "found")
num_oov, 100 * num_oov / len(doc)

virus found
virus found
virus found


(336, 4.801371820520148)

In [ ]:
# This cell tests the mechanisms used to remove stopwords,
# punctuation, spaces, and extract the token's lemma.
for token in doc:
    if not token.is_stop and not token.is_punct and not token.is_space:
        print(token.lemma_)

genetic
information
RNA
virus
organize
efficiently
Practically
nucleotide
genome
utilize
protein-coding
sequence
cis-acting
signal
translation
RNA
synthesis
RNA
encapsidation
genome
expression
strategy
group
positive-strand
RNA
+
RNA
virus
produce
subgenomic
sg
mRNAs
review
Miller
Koev
2000
replication
genomic
RNA
mRNA
viral
replicase
supplement
generation
sg
transcript
express
structural
auxiliary
protein
encode
downstream
replicase
gene
genome
Sg
mRNAs
+
RNA
virus
3′-co-terminal
genomic
RNA
different
mechanism
synthesis
virus
brome
mosaic
virus
initiate
sg
mRNA
synthesis
internally
full-length
minus
strand
RNA
template
Miller
et
al.
1985
exemplify
red
clover
necrotic
mosaic
virus
RCNMV
rely
premature
termination
minus
strand
synthesis
genomic
RNA
template
follow
synthesis
sg
plus
strand
truncate
minus
strand
template
Sit
et
al.
1998
Members
order
Nidovirales
include
coronavirus
arteriviruses
evolve
unique
mechanism
employ
discontinuous
RNA
synthesis
generation
extensive
set
sg
RNAs
r

defective
interfere
DI
RNAs
replicons
carry
body
TRSs
moderate
level
sg
mRNAs
produce
presence
helper
virus
system
Joo
Makino
1992
van
der
et
al.
1994
perform
body
TRS
mutagenesis
study
murine
coronavirus
MHV
Joo
Makino
systematically
mutagenized
core
MHV
body
TRS
contrast
result
find
21
body
TRS
mutant
sg
RNA
synthesis
DI
RNA
genome
abolish
support
normal
level
sg
RNA
production
possible
MHV
TRS
study
tolerant
single-nucleotide
mismatch
EAV
sg
RNA7
TRS
similar
study
van
der
et
al.
1994
observe
U
C
substitution
position
1
3
MHV
body
TRS
maintain
duplex
change
U
base
pair
U
G
base
pair
reduce
sg
RNA
level
strongly
substitution
disrupt
duplex
van
der
et
al.
1994
imply
case
EAV
leader
body
TRS
duplex
formation
factor
determine
coronavirus
sg
RNA
synthesis
limitation
DI
RNA
system
leader
TRS
mutagenized
study
body
TRS-specific
effect
distinguish
effect
level
leader
body
duplex
formation
recent
study
arterivirus
coronavirus
sg
RNA
synthesis
van
Marle
et
al.
1999a
Baric
Yount
2000
Sawicki
et

protein
sequence-specific
manner
body
TRS
well
candidate
serve
protein
recognition
site
protein
mediate
pause
nascent
strand
synthesis
and/or
nascent
strand
transfer
resemble
DNA-dependent
RNA
polymerase
termination
system
specific
DNA-binding
terminator
protein
bind
termination
sequence
Reeder
Lang
1997
function
HIV
nucleocapsid
protein
promote
minus
strand
strong-stop
DNA
transfer
Guo
et
al.
1997
EAV
replicase
component
nsp1
recently
show
possess
sg
RNA
synthesis-specific
activity
Tijms
et
al.
2001
good
candidate
regulatory
role
Residues
predict
form
zinc
finger
structure
nsp1
show
necessary
sg
RNA
synthesis
Interestingly
zinc
finger
structure
HIV
nucleocapsid
protein
facilitate
strand
transfer
Guo
et
al.
2000
Finally
note
RNA
structure
nascent
strand
influence
pause
strand
transfer
reinitiation
illustrate
fact
stable
hairpin
structure
nascent
strand
promote
termination
transcription
Escherichia
coli
RNA
polymerase
Wilson
von
Hippel
1995
Site-directed
mutagenesis
EAV
leader
body
TRSs

There are 95 occurrences of OOV tokens, which stands for 2.16% of the total amount of document's tokens.
Note that relevant tokens, such as `coronavirus` are included in the language model vocabulary.

## Latent Dirichlet Allocation (LDA)

The following cells will perform topic modelling experiments using the LDA technique.
The `scikit-learn` implementation of this model will be used.

In [ ]:
# First, let's process all documents texts.
def process_papers_file_contents(papers):
    texts = []
    for paper in progress_bar(papers):
        text = " \n ".join([ paragraph["text"] for paragraph in paper._file_contents["body_text"]])
        """
        NB.: for development speed purposes, the only document's attributes
        considered for the topic modelling were the title and the abstract.
        Should the text be included in other experiments, the following line
        should be modified to include `{paper.text}`.
        """
        texts.append(f"{paper.title} \n {paper.abstract}")
    return texts

docs = process_papers_file_contents(
    papers=list(pageranks.keys()),
)


In [ ]:
for doc in docs[:5]:
    print('>>', doc)

>> Sequence requirements for RNA strand transfer during nidovirus discontinuous subgenomic RNA synthesis 
 Nidovirus subgenomic mRNAs contain a leader sequence derived from the 5′ end of the genome fused to different sequences (‘bodies’) derived from the 3′ end. Their generation involves a unique mechanism of discontinuous subgenomic RNA synthesis that resembles copy-choice RNA recombination. During this process, the nascent RNA strand is transferred from one site in the template to another, during either plus or minus strand synthesis, to yield subgenomic RNA molecules. Central to this process are transcription-regulating sequences (TRSs), which are present at both template sites and ensure the fidelity of strand transfer. Here we present results of a comprehensive co-variation mutagenesis study of equine arteritis virus TRSs, demonstrating that discontinuous RNA synthesis depends not only on base pairing between sense leader TRS and antisense body TRS, but also on the primary sequenc

Vectors storing the token occurrence count will be used as document representations.
`tf-idf` vectors are purposefully not used because the document frequency normalization is already carried out by the LDA technique.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def tokenizer(sentence):
    tokens = []
    for token in nlp(sentence, disable=["tagger", "parser", "ner"]):
        # Se descartan números, stopwords, puntuación, espacio y tokens de largo 1
        if not (token.like_num or token.is_stop or token.is_punct or token.is_space or len(token)==1):
            tokens.append(token.lemma_)
    return tokens

count_vectorizer = CountVectorizer(
    tokenizer=tokenizer,
    lowercase=True,
)
vectorized_docs = count_vectorizer.fit_transform(docs)

In [ ]:
vectorized_docs

<44648x164664 sparse matrix of type '<class 'numpy.int64'>'
	with 3168863 stored elements in Compressed Sparse Row format>

In [ ]:
count_vectorizer.vocabulary_

{'sequence': 137903,
 'requirement': 130614,
 'rna': 132466,
 'strand': 143531,
 'transfer': 151049,
 'nidovirus': 105209,
 'discontinuous': 51900,
 'subgenomic': 144172,
 'synthesis': 145952,
 'mrnas': 100208,
 'contain': 44122,
 'leader': 89237,
 'derive': 50233,
 '5′': 14413,
 'end': 57299,
 'genome': 67110,
 'fuse': 65319,
 'different': 51336,
 'body': 33356,
 '3′': 12213,
 'generation': 67003,
 'involve': 83901,
 'unique': 154307,
 'mechanism': 95218,
 'resemble': 130672,
 'copy-choice': 44580,
 'recombination': 129362,
 'process': 124103,
 'nascent': 103366,
 'site': 140083,
 'template': 147705,
 'plus': 120219,
 'minus': 98084,
 'yield': 161147,
 'molecule': 99191,
 'central': 38846,
 'transcription-regulating': 150990,
 'trss': 152146,
 'present': 123413,
 'ensure': 57825,
 'fidelity': 62774,
 'result': 131035,
 'comprehensive': 43284,
 'co-variation': 42097,
 'mutagenesis': 101415,
 'study': 143846,
 'equine': 58803,
 'arteritis': 27290,
 'virus': 157408,
 'demonstrate': 49856

A sparse matrix is built with 38.882 rows, one for each document, and 149.389 columns, one for each token.

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

# Se ajusta un modelo LDA para identificar tópicos
lda = LatentDirichletAllocation(
    n_components=10,  # number of topics
    verbose=2, n_jobs=-1)
lda = lda.fit(vectorized_docs)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   49.8s remaining:   49.8s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:   52.8s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


iteration: 1 of max_iter: 10


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   32.2s remaining:   32.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:   33.8s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


iteration: 2 of max_iter: 10


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   26.6s remaining:   26.6s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:   27.9s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


...

[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   17.7s remaining:   17.7s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:   18.6s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


iteration: 9 of max_iter: 10


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   16.3s remaining:   16.3s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:   17.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


iteration: 10 of max_iter: 10


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   12.8s remaining:   12.8s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:   13.5s finished


In [ ]:
lda

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=-1,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=2)

The execution of the following cells will display the most relevant tokens for each identified topic.

In [ ]:
def print_topic_words(topic_model, vectorizer, num_words):
    feature_names = vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(topic_model.components_):
        message = "\nTopic #%d: " % topic_idx
        message += " ".join(
            [feature_names[i] for i in topic.argsort()[:-num_words - 1:-1]])
        print(message)

In [ ]:
print_topic_words(lda, count_vectorizer, 20)


Topic #0: cell response mouse immune infection expression antibody induce human disease study antigen result gene cytokine increase effect role level protein

Topic #1: patient group calve treatment study day disease clinical increase level associate effect high blood control age result lung significantly serum

Topic #2: virus protein cell viral infection rna replication host coronavirus gene antiviral activity expression sars-cov human study bind membrane domain role

Topic #3: virus strain sequence sample gene assay isolate detect coronavirus detection analysis study porcine pig test result pedv disease genome infectious

Topic #4: de nan la en et des les el los las le un se chapter que infection con une del patient

Topic #5: respiratory virus patient infection influenza child viral clinical study pneumonia acute human test result case detect sample age severe associate

Topic #6: air study effect concentration temperature result method test reduce mask aerosol activity surface tr

The dataset papers will be classified into the different previously modelled topics.

In [ ]:
docs_classified = lda.transform(vectorized_docs)
docs_classified[:5]

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   12.9s remaining:   12.9s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:   13.5s finished


array([[1.49689264e-02, 8.19863592e-04, 9.44715587e-01, 3.45768806e-02,
        8.19740956e-04, 8.19741833e-04, 8.19767263e-04, 8.19944548e-04,
        8.19776267e-04, 8.19771824e-04],
       [3.33557314e-01, 1.25001908e-03, 5.17972921e-01, 1.25009814e-03,
        1.25053607e-03, 4.61212638e-02, 1.25003355e-03, 9.48476030e-02,
        1.25011700e-03, 1.25009486e-03],
       [1.09907509e-03, 5.29271005e-02, 8.23947488e-01, 1.09928554e-03,
        1.09895796e-03, 1.09928650e-03, 1.09932241e-03, 1.02005968e-01,
        1.09919643e-03, 1.45243196e-02],
       [8.92929571e-04, 8.92916485e-04, 5.86757920e-01, 8.93033253e-04,
        8.92889038e-04, 8.92955232e-04, 5.82827871e-02, 3.48708643e-01,
        8.92947900e-04, 8.92978955e-04],
       [2.49737177e-02, 8.40394684e-04, 8.77864410e-01, 9.12787598e-02,
        8.40361663e-04, 8.40461005e-04, 8.40389795e-04, 8.40540520e-04,
        8.40479144e-04, 8.40485879e-04]])

Finalmente, para cada uno de los temas identifiados, se imprimen los top-5 papers pertenecientes al tema, ordenados por su *pagerank*.
Finally, the top-5 PageRank sorted papers belonging to each topic are displayed.

In [ ]:
docs_topics = docs_classified.argmax(1)
topic_papers = defaultdict(list)
all_papers = list(pageranks.keys())
for idx, topic_id in enumerate(docs_topics):
    topic_papers[topic_id].append(all_papers[idx])
    
for topic_id, papers in sorted(topic_papers.items(), key=lambda t: t[0]):
    print(f"Topic ID #{topic_id}")
    sorted_papers = sorted(papers, reverse=True, key=lambda p: pageranks[p])
    for paper in sorted_papers[:5]:
        paper_as_markdown(paper)
    #print("\n", end="")

Topic ID #0



- **Title:** A human in vitro model system for investigating genome-wide host responses to SARS coronavirus infection
- **Authors:** Ng, Lisa FP; Hibberd, Martin L; Ooi, Eng-Eong; Tang, Kin-Fai; Neo, Soek-Ying; Tan, Jenny; Krishna Murthy, Karuturi R; Vega, Vinsensius B; Chia, Jer-Ming; Liu, Edison T; Ren, Ee-Chee
- **Publish date/time:** 2004-09-09
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** BACKGROUND: The molecular basis of severe acute respiratory syndrome (SARS) coronavirus (CoV) induced pathology is still largely unclear. Many SARS patients suffer respiratory distress brought on by interstitial infiltration and frequently show peripheral blood lymphopenia and occasional leucopenia. One possible cause of this could be interstitial inflammation, following a localized host response. In this study, we therefore examine the immune response of SARS-CoV in human peripheral blood mononuclear cells (PBMCs) over the first 24 hours. METHODS: PBMCs from normal healthy donors were inoculated in vitro with SARS-CoV and the viral replication kinetics was studied by real-time quantitative assays. SARS-CoV specific gene expression changes were examined by high-density oligonucleotide array analysis. RESULTS: We observed that SARS-CoV was capable of infecting and replicating in PBMCs and the kinetics of viral replication was variable among the donors. SARS-CoV antibody binding assays indicated that SARS specific antibodies inhibited SARS-CoV viral replication. Array data showed monocyte-macrophage cell activation, coagulation pathway upregulation and cytokine production together with lung trafficking chemokines such as IL8 and IL17, possibly activated through the TLR9 signaling pathway; that mimicked clinical features of the disease. CONCLUSIONS: The identification of human blood mononuclear cells as a direct target of SARS-CoV in the model system described here provides a new insight into disease pathology and a tool for investigating the host response and mechanisms of pathogenesis.


- **Title:** The involvement of survival signaling pathways in rubella-virus induced apoptosis
- **Authors:** Cooray, Samantha; Jin, Li; Best, Jennifer M
- **Publish date/time:** 2005-01-04
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** Rubella virus (RV) causes severe congenital defects when acquired during the first trimester of pregnancy. RV cytopathic effect has been shown to be due to caspase-dependent apoptosis in a number of susceptible cell lines, and it has been suggested that this apoptotic induction could be a causal factor in the development of such defects. Often the outcome of apoptotic stimuli is dependent on apoptotic, proliferative and survival signaling mechanisms in the cell. Therefore we investigated the role of phosphoinositide 3-kinase (PI3K)-Akt survival signaling and Ras-Raf-MEK-ERK proliferative signaling during RV-induced apoptosis in RK13 cells. Increasing levels of phosphorylated ERK, Akt and GSK3β were detected from 24–96 hours post-infection, concomitant with RV-induced apoptotic signals. Inhibition of PI3K-Akt signaling reduced cell viability, and increased the speed and magnitude of RV-induced apoptosis, suggesting that this pathway contributes to cell survival during RV infection. In contrast, inhibition of the Ras-Raf-MEK-ERK pathway impaired RV replication and growth and reduced RV-induced apoptosis, suggesting that the normal cellular growth is required for efficient virus production.


- **Title:** Protein secretion in Lactococcus lactis : an efficient way to increase the overall heterologous protein production
- **Authors:** Le Loir, Yves; Azevedo, Vasco; Oliveira, Sergio C; Freitas, Daniela A; Miyoshi, Anderson; Bermúdez-Humarán, Luis G; Nouaille, Sébastien; Ribeiro, Luciana A; Leclercq, Sophie; Gabriel, Jane E; Guimaraes, Valeria D; Oliveira, Maricê N; Charlier, Cathy; Gautier, Michel; Langella, Philippe
- **Publish date/time:** 2005-01-04
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** Lactococcus lactis, the model lactic acid bacterium (LAB), is a food grade and well-characterized Gram positive bacterium. It is a good candidate for heterologous protein delivery in foodstuff or in the digestive tract. L. lactis can also be used as a protein producer in fermentor. Many heterologous proteins have already been produced in L. lactis but only few reports allow comparing production yields for a given protein either produced intracellularly or secreted in the medium. Here, we review several works evaluating the influence of the localization on the production yields of several heterologous proteins produced in L. lactis. The questions of size limits, conformation, and proteolysis are addressed and discussed with regard to protein yields. These data show that i) secretion is preferable to cytoplasmic production; ii) secretion enhancement (by signal peptide and propeptide optimization) results in increased production yield; iii) protein conformation rather than protein size can impair secretion and thus alter production yields; and iv) fusion of a stable protein can stabilize labile proteins. The role of intracellular proteolysis on heterologous cytoplasmic proteins and precursors is discussed. The new challenges now are the development of food grade systems and the identification and optimization of host factors affecting heterologous protein production not only in L. lactis, but also in other LAB species.


- **Title:** Expression profile of immune response genes in patients with Severe Acute Respiratory Syndrome
- **Authors:** Reghunathan, Renji; Jayapal, Manikandan; Hsu, Li-Yang; Chng, Hiok-Hee; Tai, Dessmon; Leung, Bernard P; Melendez, Alirio J
- **Publish date/time:** 2005-01-18
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** BACKGROUND: Severe acute respiratory syndrome (SARS) emerged in later February 2003, as a new epidemic form of life-threatening infection caused by a novel coronavirus. However, the immune-pathogenesis of SARS is poorly understood. To understand the host response to this pathogen, we investigated the gene expression profiles of peripheral blood mononuclear cells (PBMCs) derived from SARS patients, and compared with healthy controls. RESULTS: The number of differentially expressed genes was found to be 186 under stringent filtering criteria of microarray data analysis. Several genes were highly up-regulated in patients with SARS, such as, the genes coding for Lactoferrin, S100A9 and Lipocalin 2. The real-time PCR method verified the results of the gene array analysis and showed that those genes that were up-regulated as determined by microarray analysis were also found to be comparatively up-regulated by real-time PCR analysis. CONCLUSIONS: This differential gene expression profiling of PBMCs from patients with SARS strongly suggests that the response of SARS affected patients seems to be mainly an innate inflammatory response, rather than a specific immune response against a viral infection, as we observed a complete lack of cytokine genes usually triggered during a viral infection. Our study shows for the first time how the immune system responds to the SARS infection, and opens new possibilities for designing new diagnostics and treatments for this new life-threatening disease.

...


- **Title:** A double epidemic model for the SARS propagation
- **Authors:** Ng, Tuen Wai; Turinici, Gabriel; Danchin, Antoine
- **Publish date/time:** 2003-09-10
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** BACKGROUND: An epidemic of a Severe Acute Respiratory Syndrome (SARS) caused by a new coronavirus has spread from the Guangdong province to the rest of China and to the world, with a puzzling contagion behavior. It is important both for predicting the future of the present outbreak and for implementing effective prophylactic measures, to identify the causes of this behavior. RESULTS: In this report, we show first that the standard Susceptible-Infected-Removed (SIR) model cannot account for the patterns observed in various regions where the disease spread. We develop a model involving two superimposed epidemics to study the recent spread of the SARS in Hong Kong and in the region. We explore the situation where these epidemics may be caused either by a virus and one or several mutants that changed its tropism, or by two unrelated viruses. This has important consequences for the future: the innocuous epidemic might still be there and generate, from time to time, variants that would have properties similar to those of SARS. CONCLUSION: We find that, in order to reconcile the existing data and the spread of the disease, it is convenient to suggest that a first milder outbreak protected against the SARS. Regions that had not seen the first epidemic, or that were affected simultaneously with the SARS suffered much more, with a very high percentage of persons affected. We also find regions where the data appear to be inconsistent, suggesting that they are incomplete or do not reflect an appropriate identification of SARS patients. Finally, we could, within the framework of the model, fix limits to the future development of the epidemic, allowing us to identify landmarks that may be useful to set up a monitoring system to follow the evolution of the epidemic. The model also suggests that there might exist a SARS precursor in a large reservoir, prompting for implementation of precautionary measures when the weather cools down.


- **Title:** Air pollution and case fatality of SARS in the People's Republic of China: an ecologic study
- **Authors:** Cui, Yan; Zhang, Zuo-Feng; Froines, John; Zhao, Jinkou; Wang, Hua; Yu, Shun-Zhang; Detels, Roger
- **Publish date/time:** 2003-11-20
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** BACKGROUND: Severe acute respiratory syndrome (SARS) has claimed 349 lives with 5,327 probable cases reported in mainland China since November 2002. SARS case fatality has varied across geographical areas, which might be partially explained by air pollution level. METHODS: Publicly accessible data on SARS morbidity and mortality were utilized in the data analysis. Air pollution was evaluated by air pollution index (API) derived from the concentrations of particulate matter, sulfur dioxide, nitrogen dioxide, carbon monoxide and ground-level ozone. Ecologic analysis was conducted to explore the association and correlation between air pollution and SARS case fatality via model fitting. Partially ecologic studies were performed to assess the effects of long-term and short-term exposures on the risk of dying from SARS. RESULTS: Ecologic analysis conducted among 5 regions with 100 or more SARS cases showed that case fatality rate increased with the increment of API (case fatality = - 0.063 + 0.001 * API). Partially ecologic study based on short-term exposure demonstrated that SARS patients from regions with moderate APIs had an 84% increased risk of dying from SARS compared to those from regions with low APIs (RR = 1.84, 95% CI: 1.41–2.40). Similarly, SARS patients from regions with high APIs were twice as likely to die from SARS compared to those from regions with low APIs. (RR = 2.18, 95% CI: 1.31–3.65). Partially ecologic analysis based on long-term exposure to ambient air pollution showed the similar association. CONCLUSION: Our studies demonstrated a positive association between air pollution and SARS case fatality in Chinese population by utilizing publicly accessible data on SARS statistics and air pollution indices. Although ecologic fallacy and uncontrolled confounding effect might have biased the results, the possibility of a detrimental effect of air pollution on the prognosis of SARS patients deserves further investigation.


- **Title:** Towards evidence-based, GIS-driven national spatial health information infrastructure and surveillance services in the United Kingdom
- **Authors:** Boulos, Maged N Kamel
- **Publish date/time:** 2004-01-28
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** The term "Geographic Information Systems" (GIS) has been added to MeSH in 2003, a step reflecting the importance and growing use of GIS in health and healthcare research and practices. GIS have much more to offer than the obvious digital cartography (map) functions. From a community health perspective, GIS could potentially act as powerful evidence-based practice tools for early problem detection and solving. When properly used, GIS can: inform and educate (professionals and the public); empower decision-making at all levels; help in planning and tweaking clinically and cost-effective actions, in predicting outcomes before making any financial commitments and ascribing priorities in a climate of finite resources; change practices; and continually monitor and analyse changes, as well as sentinel events. Yet despite all these potentials for GIS, they remain under-utilised in the UK National Health Service (NHS). This paper has the following objectives: (1) to illustrate with practical, real-world scenarios and examples from the literature the different GIS methods and uses to improve community health and healthcare practices, e.g., for improving hospital bed availability, in community health and bioterrorism surveillance services, and in the latest SARS outbreak; (2) to discuss challenges and problems currently hindering the wide-scale adoption of GIS across the NHS; and (3) to identify the most important requirements and ingredients for addressing these challenges, and realising GIS potential within the NHS, guided by related initiatives worldwide. The ultimate goal is to illuminate the road towards implementing a comprehensive national, multi-agency spatio-temporal health information infrastructure functioning proactively in real time. The concepts and principles presented in this paper can be also applied in other countries, and on regional (e.g., European Union) and global levels.


- **Title:** Descriptive review of geographic mapping of severe acute respiratory syndrome (SARS) on the Internet
- **Authors:** Boulos, Maged N Kamel
- **Publish date/time:** 2004-01-28
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** From geographic mapping at different scales to location-based alerting services, geoinformatics plays an important role in the study and control of global outbreaks like severe acute respiratory syndrome (SARS). This paper reviews several geographic mapping efforts of SARS on the Internet that employ a variety of techniques like choropleth rendering, graduated circles, graduated pie charts, buffering, overlay analysis and animation. The aim of these mapping services is to educate the public (especially travellers to potentially at-risk areas) and assist public health authorities in analysing the spatial and temporal trends and patterns of SARS and in assessing/revising current control measures.


- **Title:** Recently published papers: all the usual suspects and carbon dioxide
- **Authors:** Ball, Jonathan
- **Publish date/time:** 2004-01-02
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** nan